# Импорт библиотек

In [1]:
import pickle
import random
import time
import os
import shutil
import copy

import numpy as np
import pandas as pd

from skimage import io
from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path
#import torchvision.transforms as transforms

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
#from __future__ import print_function, division
from matplotlib import colors, pyplot as plt
%matplotlib inline
plt.ion()

import torch
import torch.nn as nn
import torchvision.models as models
#import torchsummary
from torch.utils.data import datasets, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.autograd import Variable
import torch.onnx

In [17]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
EPOCHS = 40
BATCH_SIZE = 16
LEARNING_RATE = 0.001

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


# Обработка файлов

In [3]:
data_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset'
new_test = 'sum_test'
new_test_dir = os.path.join(data_dir, new_test)
os.mkdir(new_test_dir)
#new_test_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/sum_test'

In [8]:
energy_train = 'sum_train'
new_energy_dir = os.path.join(data_dir, energy_train)
os.mkdir(new_energy_dir)

In [16]:
for test_dir in tqdm(['public_test', 'private_test']):
    source_dir = os.path.join(data_dir + '/' + test_dir + '/')
    files = os.listdir(source_dir)
    for file in files:
        shutil.copyfile(source_dir, new_test_dir)

  0%|                                                    | 0/2 [00:00<?, ?it/s]


PermissionError: [Errno 13] Permission denied: 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/public_test/'

In [10]:
dir_name = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/train/'

for class_name in [1, 3, 6, 10, 20, 30]:
    os.makedirs(os.path.join(new_energy_dir, str(class_name)), exist_ok=True)

In [13]:
for type_dir in ['ER', 'NR']:
    source_dir = os.path.join(dir_name, type_dir)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if file_name[0] == '.':
            continue
        if type_dir == 'ER':
            i = 6
        else:
            i = 7
        energy = file_name.split('_')[i]

        data_save_path = new_energy_dir + '/' + energy + '/' + file_name
        shutil.copy(os.path.join(source_dir, file_name), data_save_path)

100%|██████████████████████████████████████| 6646/6646 [05:52<00:00, 18.86it/s]


# Модели

## Трансформации и загрузчики данных

In [25]:
data_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/train'

In [20]:
#Набор трансформаций для обучающей выборки
train_transforms = transforms.Compose([transforms.CenterCrop(150), 
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.RandomRotation(degrees=(45, 90)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])
                                       ])

In [21]:
#Набор трансформаций для валидационной выборки
valid_transforms = transforms.Compose([transforms.CenterCrop(150),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                       ])

In [22]:
#Набор трансформаций для тестовой выборки
test_transforms = transforms.Compose([transforms.CenterCrop(150),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                      ])

In [23]:
#Объявляю функцию загрузчика с аргументами:
#Директория, Размер батча, Набор трансформаций для тренировочной выборки,
#Набор трансформаций для валидационной выборки, Размер валидационной выборки
def load_split_train_valid(datadir,
                           batch_size,
                           train_transforms,
                           valid_transforms,
                           valid_size): 
    
    #Загрузчик для тренировочной выборки
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    #Загрузчик для тренировочной выборки
    valid_data = datasets.ImageFolder(datadir,
                    transform=valid_transforms)
    #Код для разделения на трейн и тест в указанном соотношении
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(test_idx)
    train_loader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    valid_loader = torch.utils.data.DataLoader(valid_data,
                   sampler=valid_sampler, batch_size=batch_size)
    return train_loader, valid_loader

## Модель `nn_NR_ER_v_3`

In [27]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_valid(datadir=data_dir,
                           batch_size=BATCH_SIZE,
                           train_transforms=train_transforms,
                           valid_transforms=valid_transforms,
                           valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))
train_loader.dataset.class_to_idx

['ER', 'NR']
['ER', 'NR']
671
168


{'ER': 0, 'NR': 1}

In [33]:
class ConvNetClassification(nn.Module): 
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop1 = nn.Dropout2d(p=0.25)
            self.conv2 = nn.Sequential(
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop2 = nn.Dropout2d(p=0.25)
            self.conv3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.drop3 = nn.Dropout2d(p=0.25)
            self.conv4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2)
            )
            self.fc1 = nn.Sequential(
                nn.Linear(6272, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
            )
            self.fc2 = nn.Sequential(
                nn.Linear(2048, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
            )
            self.out = nn.Linear(1024, 2)
        
        def forward(self, x):
            x = self.drop1(self.conv1(x))
            x = self.drop2(self.conv2(x))
            x = self.drop3(self.conv3(x))
            x = self.conv4(x)
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.out(x)

            return F.log_softmax(x, dim=1)

In [34]:
net_classification = ConvNetClassification()
print(net_classification)
#net = net.cuda()
net_classification = net_classification.to(device)

ConvNetClassification(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop1): Dropout2d(p=0.25, inplace=False)
  (conv2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop2): Dropout2d(p=0.25, inplace=False)
  (conv3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop3): Dropout2d(p=0.25, inplace=False)
  (conv4): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Sequential(
    (0): Linear(in_features=6272, out_features=2048, bias=True)
    (1): Batch

In [36]:
#Задаю функцию потерь
criterion = nn.NLLLoss()

#Задаю оптимизатор
optimizer = torch.optim.Adam(net_classification.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.5)

for epoch in range(EPOCHS):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for i, (images, labels) in enumerate(tqdm_notebook(train_loader)):
        # Прямой запуск
        images=images.to(device)
        labels=labels.to(device)            
        outputs = net_classification(images)
        loss = criterion(outputs, labels)

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        
        running_loss += loss.item() * total
        running_corrects += (predicted == labels).sum().item()
        processed_data += total
    scheduler.step()

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, EPOCHS, running_loss / processed_data,
                  (running_corrects / processed_data) * 100))

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/671 [00:00<?, ?it/s]

Epoch [1/40], Loss: 0.7446, Accuracy: 49.57%


  0%|          | 0/671 [00:00<?, ?it/s]

Epoch [2/40], Loss: 0.7123, Accuracy: 49.69%


...

Epoch [38/40], Loss: 0.0050, Accuracy: 99.89%


  0%|          | 0/671 [00:00<?, ?it/s]

Epoch [39/40], Loss: 0.0090, Accuracy: 99.74%


  0%|          | 0/671 [00:00<?, ?it/s]

Epoch [40/40], Loss: 0.0054, Accuracy: 99.82%


### Проверка на валидационной выборке

In [38]:
#Проверка на валидационной выборке
net_classification.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(val_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net_classification(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

100%|████████████████████████████████████████| 168/168 [02:09<00:00,  1.30it/s]

Test Accuracy of the model on the test images: 99.77611940298507 %


### Обучение модели на всей тренировочной выборке

In [40]:
train_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/train/'

In [41]:
train_data = datasets.ImageFolder(train_dir,
                transform=train_transforms)

full_train_loader = DataLoader(train_data, shuffle=False, batch_size=BATCH_SIZE)

In [ ]:
#Задаю функцию потерь
criterion = nn.NLLLoss()

#Задаю оптимизатор
optimizer = torch.optim.Adam(net_classification.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.5)

for epoch in range(EPOCHS):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for i, (images, labels) in enumerate(tqdm_notebook(full_train_loader)):
        # Прямой запуск
        images=images.to(device)
        labels=labels.to(device)            
        outputs = net_classification(images)
        loss = criterion(outputs, labels)

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        
        running_loss += loss.item() * total
        running_corrects += (predicted == labels).sum().item()
        processed_data += total
    scheduler.step()

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, EPOCHS, running_loss / processed_data,
                  (running_corrects / processed_data) * 100))

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/838 [00:00<?, ?it/s]

Epoch [1/40], Loss: 0.0421, Accuracy: 99.83%


  0%|          | 0/838 [00:00<?, ?it/s]

Epoch [2/40], Loss: 0.8770, Accuracy: 97.72%


...

  0%|          | 0/838 [00:00<?, ?it/s]

Epoch [38/40], Loss: 0.5062, Accuracy: 78.30%


  0%|          | 0/838 [00:00<?, ?it/s]

Epoch [39/40], Loss: 0.4638, Accuracy: 80.38%


  0%|          | 0/838 [00:00<?, ?it/s]

### Сохранение модели

In [ ]:
torch.save(net_classification, 'D:/_data_scienсe/IDAO_2021/models/IDAO_nn_NR_ER_v_3.pt')

### Сабмит модели `nn_NR_ER_v_3`

In [ ]:
test_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/sum_test'

In [ ]:
test_data = datasets.ImageFolder(test_dir,
                transform=test_transforms)

test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)
filenames = [line[0].split('\\')[1].split('.')[0] for line in test_data.imgs]
dict_pred = defaultdict(list)
dict_pred["id"] = filenames

In [ ]:
%%time
net_classification.eval()
with torch.no_grad():
    for i, (img, name) in tqdm(enumerate(iter(test_loader))):
        img = img.to(device)
        outputs = net_classification(img)
        _, predicted = torch.max(outputs.data, 1)
        predicted = list(map(int, predicted.cpu()))
        dict_pred["classification_predictions"].extend(predicted)
print(len(dict_pred['id']))
print(len(dict_pred['classification_predictions']))

In [ ]:
data_frame = pd.DataFrame(dict_pred, columns=["id", "classification_predictions"])
data_frame['classification_predictions'] = data_frame['classification_predictions'].replace({0: 1, 1: 0})
data_frame.to_csv('D:/_data_scienсe/IDAO_2021/tarck_a/submission/IDAO_class_submission.csv', index=False, header=True)

## Модель `nn_ENERGY_v_3`

In [ ]:
data_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/sum_train'

In [ ]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_valid(datadir=data_dir,
                           batch_size=BATCH_SIZE,
                           train_transforms=train_transforms,
                           valid_transforms=valid_transforms,
                           valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))

In [ ]:
class ConvNetRegression(nn.Module): 
        def __init__(self):
            super().__init__()
            self.conv1 = nn.Sequential(
                nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
            )
            self.drop1 = nn.Dropout2d(p=0.25)
            self.conv2 = nn.Sequential(
                nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
            )
            self.drop2 = nn.Dropout2d(p=0.25)
            self.conv3 = nn.Sequential(
                nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
                nn.BatchNorm2d(128),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
            )
            self.drop3 = nn.Dropout2d(p=0.25)
            self.conv4 = nn.Sequential(
                nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                nn.BatchNorm2d(128),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=2),
            )
            self.fc1 = nn.Sequential(
                nn.Linear(6272, 2048),
                nn.BatchNorm1d(2048),
                nn.ReLU(),
            )
            self.fc2 = nn.Sequential(
                nn.Linear(2048, 1024),
                nn.BatchNorm1d(1024),
                nn.ReLU(),
            )
            self.out = nn.Linear(1024, 6)
        
        def forward(self, x):
            x = self.drop1(self.conv1(x))
            x = self.drop2(self.conv2(x))
            x = self.drop3(self.conv3(x))
            x = self.conv4(x)
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            x = self.fc2(x)
            x = self.out(x)

            return F.log_softmax(x, dim=1)

In [ ]:
net_regression = ConvNetRegression()
print(net_regression)
#net = net.cuda()
net_regression = net_regression.to(device)

In [ ]:
#Задаю функцию потерь
criterion = nn.NLLLoss()


#Задаю оптимизатор
optimizer = torch.optim.Adam(net_regression.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.5)

for epoch in range(EPOCHS):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for i, (images, labels) in enumerate(tqdm_notebook(train_loader)):
        #Прямой запуск
        images=images.to(device)
        labels=labels.to(device)            
        outputs = net_regression(images)
        loss = criterion(outputs, labels)

        #Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        
        running_loss += loss.item() * total
        running_corrects += (predicted == labels).sum().item()
        processed_data += total
    scheduler.step()

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, EPOCHS, running_loss / processed_data,
                  (running_corrects / processed_data) * 100))

### Проверка на валидационной выборке

In [ ]:
#Проверка на валидационной выборке
net_regression.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(val_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net_regression(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

### Обучение модели на всей тренировочной выборке

In [ ]:
train_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/sum_train/'

In [ ]:
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
full_train_loader = DataLoader(train_data, shuffle=False, batch_size=BATCH_SIZE)

In [ ]:
#Задаю функцию потерь
criterion = nn.NLLLoss()

#Задаю оптимизатор
optimizer = torch.optim.Adam(net_regression.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.5)

for epoch in range(EPOCHS):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
    for i, (images, labels) in enumerate(tqdm_notebook(full_train_loader)):
        # Прямой запуск
        images=images.to(device)
        labels=labels.to(device)            
        outputs = net_regression(images)
        loss = criterion(outputs, labels)

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        
        running_loss += loss.item() * total
        running_corrects += (predicted == labels).sum().item()
        processed_data += total
    scheduler.step()

    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch + 1, EPOCHS, running_loss / processed_data,
                  (running_corrects / processed_data) * 100))

### Сохранение модели

In [ ]:
torch.save(net_regression, 'D:/_data_scienсe/IDAO_2021/models/IDAO_nn_ENERGY_v_3.pt')

### Сабмит модели `nn_ENERGY_v_3`

In [ ]:
test_dir = 'D:/_data_scienсe/IDAO_2021/idao_dataset/_new_dataset/sum_test'

In [ ]:
test_data = datasets.ImageFolder(test_dir,
                transform=test_transforms)

test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)
filenames = [line[0].split('\\')[1].split('.')[0] for line in test_data.imgs]
pred2class = [float(n) for n in train_loader.dataset.classes]
dict_pred = defaultdict(list)
dict_pred["id"] = filenames

In [ ]:
%%time
net_regression.eval()
with torch.no_grad():
    for i, (img, name) in enumerate(iter(test_loader)):
        img = img.to(device)
        outputs = net_regression(img)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu()
        predicted = [pred2class[pred] for pred in predicted]
        dict_pred["regression_predictions"].extend(predicted)

In [ ]:
data_frame = pd.DataFrame(dict_pred, columns=["id", "regression_predictions"])
data_frame.to_csv('D:/_data_scienсe/IDAO_2021/tarck_a/submission/IDAO_regr_submission.csv', index=False, header=True)

# Суммарный сабмит моделей

In [ ]:
cls = pd.read_csv('D:/_data_scienсe/IDAO_2021/tarck_a/submission/IDAO_class_submission.csv')
reg = pd.read_csv('D:/_data_scienсe/IDAO_2021/tarck_a/submission/IDAO_regr_submission.csv')

In [ ]:
cls['regression_predictions'] = reg['regression_predictions']
cls.to_csv('D:/_data_scienсe/IDAO_2021/tarck_a/submission/IDAO_submission.csv', index=False, header=True)